<a href="https://colab.research.google.com/github/MinahilDar011/web-work/blob/main/TOPLproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile

In [ ]:
!file /content/drive/MyDrive/effibench_dataset

/content/drive/MyDrive/effibench_dataset: directory


In [ ]:
!ls

drive  sample_data


In [ ]:
!ls /content/drive/MyDrive/effibench_dataset

 aizu_1444_yokohama-phenomena.json
 aizu_1447_nested-repetition-compression.json
 aizu_1448_chayas.json
 aizu_1449_color-inversion-on-a-huge-chessboard.json
 aizu_1455_ribbon-on-the-christmas-present.json
 aizu_1456_the-sparsest-number-in-between.json
'aizu_1459_e-circuit-is-now-on-sale!.json'
 aizu_1460_the-farthest-point.json
 aizu_1464_mixing-solutions.json
 aizu_1664_which-team-should-receive-the-sponsor-prize_.json
 aizu_1665_amidakuji.json
 aizu_1666_changing-the-sitting-arrangement.json
 aizu_1668_tampered-records.json
 aizu_1672_snacks-within-300-yen.json
 aizu_1673_overtaking.json
 aizu_1674_honeycomb-distance.json
 aizu_1678_two-sets-of-cards.json
 aizu_3600_sum-of-product-of-binomial-coefficients.json
 aizu_3612_roller-coaster.json
 aizu_3613_break-a-prison.json
 aizu_3614_camp-room-assignment.json
 aizu_3616_gacha-101.json
 aizu_3619_lcp-queries.json
 aizu_3627_expression-sum.json
 aizu_3629_give-me-a-lot-of-triangles.json
 aizu_3630_half-plane-painting.json
 aizu_3631_i-lo

In [ ]:
!pip install pandas scikit-learn

In [ ]:
import os
import json
import pandas as pd

In [ ]:
DATASET_FOLDER = "/content/drive/MyDrive/effibench_dataset"

all_data = []
file_count = 0

for root, dirs, files in os.walk(DATASET_FOLDER):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                all_data.append(data)

            file_count += 1
            if file_count % 100 == 0:
                print(f"Loaded {file_count} files...")

print("Total JSON files loaded:", len(all_data))

Loaded 100 files...
Loaded 200 files...
Loaded 300 files...
Loaded 400 files...
Loaded 500 files...
Loaded 600 files...
Total JSON files loaded: 623


In [ ]:
java_data = []

for item in all_data:
    solutions = item.get("solutions", {})
    if "java" in solutions:
        java_data.append(item)

print("Java code snippets:", len(java_data))

Java code snippets: 623


In [ ]:
java_codes = []

for item in all_data:
    java_solution = item.get("solutions", {}).get("java")
    if java_solution and "code" in java_solution:
        java_codes.append(java_solution["code"])

print("Java code snippets:", len(java_codes))

Java code snippets: 623


In [ ]:
def extract_features(code):
    lines = code.split("\n")
    loc = len(lines)
    loops = code.count("for") + code.count("while")
    return loc, loops

In [ ]:
features = []
labels = []

for idx, item in enumerate(java_data):
    solutions = item.get("solutions")
    if not solutions:
        continue

    java_solutions = solutions.get("java")
    if not java_solutions:
        continue

    # Normalize java_solutions to a list
    if isinstance(java_solutions, dict):
        java_solutions = [java_solutions]

    for sol in java_solutions:  # now always safe
        code = sol.get("code")
        if not code:
            continue

        runtime = sol.get("runtime", 0)

        loc, loops = extract_features(code)
        features.append([loc, loops])
        labels.append(runtime)

print(f"Collected {len(features)} Java samples")

if features:
    df = pd.DataFrame(features, columns=["Lines_of_Code", "Loop_Count"])
    df["Runtime"] = labels
    print(df.head())

Collected 623 Java samples
   Lines_of_Code  Loop_Count  Runtime
0            384          29    224.0
1            318          17     39.0
2            471          30    373.0
3            370          28    134.0
4             97           4     41.0


In [ ]:
from sklearn.model_selection import train_test_split

# Step 1: Drop rows where Runtime is missing
df = df.dropna(subset=["Runtime"])

# Step 2: Extract features and target
X = df[["Lines_of_Code", "Loop_Count"]]
y = df["Runtime"]

# Optional sanity check to ensure no NaNs remain
print("Missing values in X:\n", X.isna().sum())
print("Missing values in y:", y.isna().sum())

# Step 3: Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")

Missing values in X:
 Lines_of_Code    0
Loop_Count       0
dtype: int64
Missing values in y: 0
Training samples: 480, Test samples: 120


In [ ]:
from sklearn.linear_model import LinearRegression

# Step 4: Train Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

print("Model trained successfully!")

Model trained successfully!


In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = model.predict(X_test)
error = mean_absolute_error(y_test, predictions)

print("Average prediction error:", error)

Average prediction error: 234.10412148357744


In [ ]:
new_java_code = """
public class Test {
    public static void main(String[] args) {
        for(int i=0;i<10;i++){
            System.out.println(i);
        }
    }
}
"""

loc, loops = extract_features(new_java_code)

predicted_runtime = model.predict([[loc, loops]])

print("Predicted runtime:", predicted_runtime[0])

Predicted runtime: 190.67146333100578


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
